In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns
import acquire
from acquire import get_codeup_data

# Goal of this project is to answer a set of questions regarding the Codeup dataset that will be delivered to Maggie via email, which she will use as part of a presentation in a board meeting. 

-------------------------------------------------------------

**First thing is to acquire the data from the acquire.py file**

In [2]:
df = get_codeup_data()